# Generators: x = yield 42 
## And other applications

### David Stuebe

<a href="https://www.swipely.com/" target="_blank">www.swipely.com</a>

<a href="mailto:davidstuebe@swipely.com">davidstuebe@swipely.com</a>

March 2, 2015

### Based on David M. Beazley
### Generators: The Final Frontier
<a href="http://www.dabeaz.com/finalgenerator/" target="_blank">http://www.dabeaz.com/finalgenerator/</a>

### A Curious Course on Coroutines and Concurrency
<a href="http://www.dabeaz.com/coroutines/" target="_blank">http://www.dabeaz.com/coroutines/</a>


## What is a generator?
```
    A kind of function that can return an intermediate result ("the next
    value") to its caller, but maintaining the function's local state so
    that the function can be resumed again right where it left off.
```
<a href="https://www.python.org/dev/peps/pep-0255/" target="_blank">https://www.python.org/dev/peps/pep-0255//</a>



In [30]:
def fib():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a+b

In [31]:
g = fib()
print g


<generator object fib at 0x111227e60>


In [32]:
print g.next()
print g.next()
print g.next()
print g.next()
print g.next()

1
1
2
3
5


In [33]:
def coroutine(func):
  """ A helper function decorator from Beazley"""
  def start(*args, **kwargs):
    g = func(*args, **kwargs)
    g.next()
    return g
  return start

@coroutine
def cotuple2list():
  """This does the work"""
  result = None
  while True:
    (tup, co_pool) = (yield result)
    result = list(tup)
    # I don't like using append. So I am changing the data in place.
    for (i,x) in enumerate(result):
      # consider using "if hasattr(x,'__iter__')"
      if isinstance(x,tuple):
        result[i] = co_pool[0].send((x, co_pool[1:]))


@coroutine
def colist2tuple():
  """This does the work"""
  result = None
  while True:
    (lst, co_pool) = (yield result)
    # I don't like using append so I am changing the data in place...
    for (i,x) in enumerate(lst):
      # consider using "if hasattr(x,'__iter__')"
      if isinstance(x,list):
        lst[i] = co_pool[0].send((x, co_pool[1:]))
    result = tuple(lst)

def list2tuple(a):
  return tuple((list2tuple(x) if isinstance(x, list) else x for x in a))

def tuple2list(a):
  return list((tuple2list(x) if isinstance(x, tuple) else x for x in a))


def make_test(m, n):
  # Test data function taken from HYRY's post!
  return [[range(m), make_test(m, n-1)] for i in range(n)]


In [34]:
list2tuple(make_test(2,3))

(((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))),
 ((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))),
 ((0, 1), (((0, 1), (((0, 1), ()),)), ((0, 1), (((0, 1), ()),)))))